# Model Signature

## Column Based Signature

In [1]:
import mlflow

In [2]:
experiment_name = "model-signature"
try:
    experiment_id = mlflow.create_experiment(experiment_name)
except Exception:
    experiment = mlflow.get_experiment_by_name(experiment_name)

experiment = mlflow.set_experiment(experiment_name)

## Defining a signature

In [3]:
from mlflow.models import ModelSignature
from mlflow.types.schema import Schema 
from mlflow.types.schema import ColSpec

from sklearn.datasets import load_iris

In [4]:
# loading the iris dataset
iris = load_iris(as_frame=True)
iris.data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [5]:
# features
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [6]:
# schema for the feature columns
col = ColSpec(type="double", name="sepal length (cm)", required=True)
col.to_dict()

{'type': 'double', 'name': 'sepal length (cm)', 'required': True}

In [7]:
col_specifications = [
    ColSpec(type="double", name=feature_name, required=True) for feature_name in iris.feature_names
    ]
model_input = Schema(inputs = col_specifications)

In [8]:
model_output = Schema(inputs= [ColSpec(type="integer", name="species", required=True)])

In [9]:
model_output.to_dict()
model_input.to_dict()

[{'type': 'double', 'name': 'sepal length (cm)', 'required': True},
 {'type': 'double', 'name': 'sepal width (cm)', 'required': True},
 {'type': 'double', 'name': 'petal length (cm)', 'required': True},
 {'type': 'double', 'name': 'petal width (cm)', 'required': True}]

In [10]:
# creating the model signature
model_signature = ModelSignature(
    inputs = model_input,
    outputs= model_output
)


In [11]:
from pprint import pprint

pprint(model_signature.to_dict(), indent=2)


{ 'inputs': '[{"type": "double", "name": "sepal length (cm)", "required": '
            'true}, {"type": "double", "name": "sepal width (cm)", "required": '
            'true}, {"type": "double", "name": "petal length (cm)", '
            '"required": true}, {"type": "double", "name": "petal width (cm)", '
            '"required": true}]',
  'outputs': '[{"type": "integer", "name": "species", "required": true}]',
  'params': None}


In [12]:
model_signature.inputs.to_dict()

[{'type': 'double', 'name': 'sepal length (cm)', 'required': True},
 {'type': 'double', 'name': 'sepal width (cm)', 'required': True},
 {'type': 'double', 'name': 'petal length (cm)', 'required': True},
 {'type': 'double', 'name': 'petal width (cm)', 'required': True}]

In [13]:
model_signature.outputs.to_dict()

[{'type': 'integer', 'name': 'species', 'required': True}]

In [14]:
model_signature.params

## Adding Parameters

In [15]:
from mlflow.models.signature import ParamSchema
from mlflow.types.schema import ParamSpec

In [16]:
param1 = ParamSpec(name="region", dtype="string",default="USA")
param2 = ParamSpec(name="temperature", dtype="double", default=0.0)

In [17]:
param_schema = ParamSchema(params=[param1, param2])
param_schema.to_dict()

[{'name': 'region', 'default': 'USA', 'shape': None, 'type': 'string'},
 {'name': 'temperature', 'default': 0.0, 'shape': None, 'type': 'double'}]

In [18]:
model_signature_with_params = ModelSignature(inputs=model_input, outputs = model_output, params=param_schema)
pprint(model_signature_with_params.to_dict())

{'inputs': '[{"type": "double", "name": "sepal length (cm)", "required": '
           'true}, {"type": "double", "name": "sepal width (cm)", "required": '
           'true}, {"type": "double", "name": "petal length (cm)", "required": '
           'true}, {"type": "double", "name": "petal width (cm)", "required": '
           'true}]',
 'outputs': '[{"type": "integer", "name": "species", "required": true}]',
 'params': '[{"name": "region", "default": "USA", "shape": null, "type": '
           '"string"}, {"name": "temperature", "default": 0.0, "shape": null, '
           '"type": "double"}]'}


### Example: Login model using schema

In [19]:
from sklearn.ensemble import RandomForestClassifier 

rfc = RandomForestClassifier()
rfc.fit(iris.data, iris.target)

with mlflow.start_run(run_name="model_signature") as run:
    mlflow.sklearn.log_model(rfc, "rfc", signature=model_signature_with_params)

In [20]:
my_model = mlflow.sklearn.load_model("runs:/"+run.info.run_id+"/rfc")
my_model.predict(iris.data)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Parameters are only accepted when the model is loaded back using pyfunc flavor.

In [21]:
my_model_py = mlflow.pyfunc.load_model("runs:/"+run.info.run_id+"/rfc")
my_model_py.predict(iris.data, params={"region":"USA", "temperature":0.0})

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### Data Validation

In [22]:
new_data = iris.data.copy()
# changing the data type of the first column
new_data["sepal length (cm)"] = new_data["sepal length (cm)"].astype(str)

try:
    predictions = my_model.predict(new_data)
except Exception as e:
    print(e)

In [23]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])